# CS5100 Final Project

Name: Your Name Here

Goal: This assignment will allow you to apply your knowledge of machine learning that you have aquired in this course on a real-world dataset of your choosing per the assignment specifications.

Complete the notebook and demonstrate your mastery of the topic.

## Prerequisites

In this section describe any packages that need to be installed or any additional setup that needs to take place before trying to run code.

## Problem

In this section, you will explain the problem you are trying to solve, the things you are trying to learn, and how you want to visualize the results.

## Data

In this section you will describe the dataset (corpus) you are using. Explain who collected the data, how they did it, the manner in which it was stored, the way you accessed the data, what steps you will have to do to clean it, and so on. Provide any relavent details.

## Cleaning the Data

Add the code to download/query you data. If your data is already downloaded, then load it into your program here. Then add the code to clean it.

## Learning Model

In detailed and documented Python, walk the user through your code below similar to the Twitter assignment.

## Tuning and Feature Extraction

Add and describe code to perform Hyperparameter Tuning and Feature Extraction or PCA.

## Visuals

Below, generate the visuals that will explain the results you are getting and show what you hoped to learn from the data.

## Refection

1. What issues did you encounter when aquiring and cleaning the data?
2. What features did you use in your final model and why were others excluded?
3. Which ML methods did you use and why? Which performed best before you started tuning? Did things change after tuning?
4. What was your accuracy before and then after cross validation?
5. What was the most challenging part of this project?